<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective"><strong>Objective</strong></h3>
<strong> This runbook demonstrates how to enforce HTTP redirection across AWS ALB using unSkript actions.</strong></div>
</center><center>
<h2 id="Enforce-HTTP-Redirection-Across-AWS-ALB">Enforce HTTP Redirection Across AWS ALB</h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1.&nbsp;Get AWS ALB Listeners Without HTTP Redirection.<br>2.&nbsp;AWS Modify ALB Listeners HTTP Redirection.</p>

In [ ]:
if alb_listener_arns and not region:
    raise SystemExit("Enter region for given ALB Listener's!")
if region == None:
    region = ""

<h3 id="Get-AWS-ALB-Listeners-Without-HTTP-Redirection">Get AWS ALB Listeners Without HTTP Redirection</h3>
<p>Here we will use unSkript <strong>Get AWS ALB Listeners Without HTTP Redirection</strong> action. In this action, we will check for listener configuration for HTTP redirection and return a list of listener ARNs that don't have HTTP redirection.</p>
<blockquote>
<p>Input parameters: <code>region</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>listener_arns</code></p>
</blockquote>

In [ ]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import Optional, Tuple
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
from unskript.legos.aws.aws_list_application_loadbalancers.aws_list_application_loadbalancers import aws_list_application_loadbalancers
import pprint


from beartype import beartype
@beartype
def aws_get_alb_listeners_without_http_redirect_printer(output):
    if output is None:
        return

    pprint.pprint(output)


@beartype
def aws_get_alb_listeners_without_http_redirect(handle, region: str = "") -> Tuple:
    """aws_get_alb_listeners_without_http_redirect List of ALB listeners without HTTP redirection.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: Region to filter ALB listeners.

        :rtype: Tuple of status result and list of ALB listeners without HTTP redirection.
    """
    result = []
    all_regions = [region]
    alb_list = []
    if not region:
        all_regions = aws_list_all_regions(handle)

    for reg in all_regions:
        try:
            alb_dict = {}
            loadbalancer_arn = aws_list_application_loadbalancers(handle, reg)
            alb_dict["region"] = reg
            alb_dict["alb_arn"] = loadbalancer_arn
            alb_list.append(alb_dict)
        except Exception as error:
            pass

    for alb in alb_list:
        try:
            ec2Client = handle.client('elbv2', region_name=alb["region"])
            for load in alb["alb_arn"]:
                response = aws_get_paginator(ec2Client, "describe_listeners", "Listeners",
                                             LoadBalancerArn=load)
                for listner in response:
                    if 'SslPolicy' not in listner:
                        resp = aws_get_paginator(ec2Client, "describe_rules", "Rules",
                                             ListenerArn=listner['ListenerArn'])
                        for rule in resp:
                            for action in rule['Actions']:
                                listener_dict = {}
                                if action['Type'] != 'redirect':
                                    listener_dict["region"] = alb["region"]
                                    listener_dict["listener_arn"] = listner['ListenerArn']
                                    result.append(listener_dict)
        except Exception as error:
            pass

    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)



task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not alb_listener_arns",
    "condition_result": true
    }''')

task.configure(outputName="listener_arns")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_alb_listeners_without_http_redirect, lego_printer=aws_get_alb_listeners_without_http_redirect_printer, hdl=hdl, args=args)

<h3 id="Modify-Listener-ARNs-Output">Modify Listener ARNs Output</h3>
<p>In this action, we modify the output from step 2 and return a list of dictionary items for the Listener's ARNs.</p>
<blockquote>
<p>Output variable: <code>arn_list</code></p>
</blockquote>

In [22]:
import re 
import json
from unskript.legos.utils import parseARN


arns_list = []
try:
    if listener_arns[0] == False:
        for listener in listener_arns[1]:
            arns_list.append(listener)
except Exception as e:
    if alb_listener_arns:
        for i in alb_listener_arns:
            arn_dict = {}
            parsedArn = parseARN(i)
            arn_dict["region"] = parsedArn["region"]
            arn_dict["listener_arn"] = i
            arns_list.append(arn_dict)
    else:
        raise Exception(e)

<h3 id="AWS-Modify-ALB-Listeners-HTTP-Redirection"><a id="2" target="_self" rel="nofollow"></a>AWS Modify ALB Listeners HTTP Redirection</h3>
<p>Here we will use unSkript <strong>AWS Modify ALB Listeners HTTP Redirection</strong> action. In this action, we will modify a listener's configuration for HTTP redirection to the listener, which we get from step 2. This action only executes when len(Listener_ARNs)&gt;0.</p>
<blockquote>
<p>Input parameters: <code>listener_arn</code>, <code>region</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>Modified_Output</code></p>
</blockquote>

In [6]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint

from beartype import beartype
@beartype
def aws_modify_listener_for_http_redirection_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_modify_listener_for_http_redirection(handle, listener_arn: str, region: str) -> List:
    """aws_modify_listener_for_http_redirection List of Dict with modified listener info.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type listener_arn: string
        :param listener_arn: List of LoadBalancerArn.

        :type region: string
        :param region: Region to filter ALB listeners.

        :rtype: List of Dict with modified ALB listeners info.
    """
    listner_config = [{
                        "Type": "redirect",
                        "Order": 1,
                        "RedirectConfig": {
                            "Protocol": "HTTPS",
                            "Host": "#{host}",
                            "Query": "#{query}",
                            "Path": "/#{path}",
                            "Port": "443",
                            "StatusCode": "HTTP_302"}}]
    result = []
    try:
        #if ALB_Name in listener_arn:
        ec2Client = handle.client('elbv2', region_name=region)
        response = ec2Client.modify_listener(ListenerArn=listener_arn,
                                                 DefaultActions=listner_config)
        result.append(response)

    except Exception as error:
        result.append(error)

    return result


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "iter.get(\\"region\\")",
    "listener_arn": "iter.get(\\"listener_arn\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "arns_list",
    "iter_parameter": ["listener_arn","region"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(arns_list)>0",
    "condition_result": true
    }''')
task.configure(outputName="modified_output")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_modify_listener_for_http_redirection, lego_printer=aws_modify_listener_for_http_redirection_printer, hdl=hdl, args=args)

### Conclusion
<p>In this Runbook, we demonstrated the use of unSkript's AWS actions and this runbook find out all the Application Load Balancer listeners without HTTP redirection and modify them for HTTP redirection. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">https://us.app.unskript.io</a></p>